In [ ]:
import pandas as pd
import pandas
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LassoLarsCV
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
import numpy as np
from xgboost import XGBRegressor
import xgboost as xgb

In [ ]:
working_directory = '/content/drive/MyDrive/Arrow Electronics MSBC5490'

In [ ]:
url = '/content/drive/MyDrive/Arrow Electronics MSBC5490/LEADTIME_unhashed.csv'
arrow_DF = pd.read_csv(url)
arrowDF = pd.DataFrame(arrow_DF)

arrowDF = arrow_DF.drop(columns = 'Unnamed: 0')

## Setup and Inspect the Data

In [ ]:
frequency = arrowDF['mfr'].value_counts()
frequency

mfr_item10     8886
mfr_item4      3126
mfr_item0      2998
mfr_item17     2606
mfr_item9      2427
               ... 
mfr_item250       1
mfr_item296       1
mfr_item218       1
mfr_item237       1
mfr_item252       1
Name: mfr, Length: 337, dtype: int64

In [ ]:
frequency = pd.DataFrame(frequency)
sum(frequency['mfr'].head(18))/sum(frequency['mfr'])

0.6035400930800335

In [ ]:
frequencyList = list(frequency.head(18).index)
frequencyList

['mfr_item10',
 'mfr_item4',
 'mfr_item0',
 'mfr_item17',
 'mfr_item9',
 'mfr_item3',
 'mfr_item7',
 'mfr_item40',
 'mfr_item6',
 'mfr_item103',
 'mfr_item37',
 'mfr_item16',
 'mfr_item30',
 'mfr_item19',
 'mfr_item69',
 'mfr_item14',
 'mfr_item18',
 'mfr_item27']

In [ ]:
arrowDF2 = arrowDF[arrowDF['mfr'].isin(frequencyList)]
arrowDF2.head()

,mfr,icc3,icc2,icc1,expected_leadtime,actualleadtime,mpn,PCA0,PCA1,PCA2,PCA3,PCA4,PCA5,PCA6,PCA7,PCA8,PCA9,PCA10,PCA11,PCA12,PCA13,PCA14,PCA15,PCA16,PCA17,PCA18,PCA19,PCA20,PCA21,PCA22,PCA23,PCA24,PCA25,PCA26,PCA27,PCA28,PCA29,PCA30,PCA31,PCA32,PCA33,PCA34,PCA35,PCA36,PCA37,PCA38,PCA39,PCA40,PCA41
0,mfr_item0,icc3_item0,icc2_item0,icc1_item0,22,15,mpn_item0,-1.174946,-0.403428,0.810248,0.215991,0.003821,-0.065177,0.205518,0.299902,-0.040518,-0.007413,-0.080160,-0.017602,-0.113835,-0.182175,-0.045154,0.146770,0.041436,-0.004824,-0.069806,0.095916,-0.024806,0.014732,0.241382,0.130236,0.021880,0.049609,0.038993,0.009823,0.034843,-0.062368,-0.038443,0.054939,-0.020105,-0.003368,-0.018590,1.913301e-16,-2.232411e-17,6.222737e-17,-3.373274e-17,-7.943787e-17,2.051449e-17,9.597281e-17
1,mfr_item0,icc3_item0,icc2_item0,icc1_item0,20,8,mpn_item1,-1.330728,-0.375666,0.734416,0.218536,0.007240,-0.175244,0.049042,0.099806,-0.100697,0.024944,-0.373073,-0.045814,-0.056477,-0.887104,-0.104143,0.407433,0.207123,0.269722,-0.021335,-0.052379,0.080534,-0.082654,-0.009491,0.098660,0.038703,0.051741,-0.013583,0.014525,-0.002262,-0.070955,-0.046545,-0.009958,-0.003578,-0.008241,0.005415,6.653302e-17,3.992338e-17,2.284957e-17,-1.007732e-16,4.666063e-17,6.117001e-17,7.838461e-17
2,mfr_item0,icc3_item0,icc2_item0,icc1_item0,29,24,mpn_item2,-1.408653,-0.358471,0.745194,0.231599,0.008366,-0.121095,0.170981,0.445013,0.024451,-0.026001,0.169235,-0.017426,0.020527,0.145947,-0.005541,-0.048399,0.072314,0.109221,-0.022875,0.012557,0.061666,-0.087305,0.027153,0.046812,0.005431,0.004827,0.010242,0.009830,0.009564,0.000274,-0.018409,0.009750,0.004481,-0.004457,0.007975,1.195541e-17,-8.070977e-18,2.878232e-17,-1.358620e-17,-2.305384e-17,4.956489e-17,5.866807e-18
3,mfr_item0,icc3_item0,icc2_item0,icc1_item0,29,14,mpn_item3,-1.220221,-0.376939,0.756570,0.214228,0.005685,-0.131327,0.157541,0.206358,-0.051941,0.005890,-0.225218,-0.032813,-0.129289,-0.537969,-0.065728,0.280753,0.125759,0.118543,-0.057094,0.063039,0.022790,-0.037352,0.141052,0.134637,0.031110,0.006709,0.005382,0.006348,0.009572,-0.011692,-0.018752,0.022924,-0.014351,-0.006275,0.001166,2.358487e-16,6.590460e-18,4.027744e-17,-6.463125e-17,-1.102957e-17,5.464567e-18,8.719049e-17
6,mfr_item0,icc3_item0,icc2_item0,icc1_item0,20,2,mpn_item6,-1.433350,-0.359800,0.766970,0.237394,0.006382,-0.080985,0.174022,0.401849,0.069397,-0.025626,0.111609,-0.009596,-0.117677,0.158214,0.023140,-0.047655,0.039355,0.054083,-0.030173,0.015012,0.041469,-0.068194,0.007314,0.039451,0.011591,0.043217,0.010300,0.011485,0.009609,-0.055956,-0.038019,-0.014139,0.010816,-0.007333,0.007349,-5.743938e-17,8.551771e-18,3.582364e-17,-1.238643e-17,-6.659102e-17,4.263336e-17,-4.552081e-18


In [ ]:
print(len(arrowDF2['mfr'].unique()))
print(len(arrowDF2['icc3'].unique()))
print(len(arrowDF2['icc2'].unique()))
print(len(arrowDF2['icc1'].unique()))

18
362
50
10


In [ ]:
print(len(arrowDF['mfr'].unique()))
print(len(arrowDF['icc3'].unique()))
print(len(arrowDF['icc2'].unique()))
print(len(arrowDF['icc1'].unique()))

337
506
62
10


## Select Features and Target, Test and Split Data 

In [ ]:
features_imp = ['mfr','icc1','icc2','PCA0','PCA1','PCA3','PCA4','PCA5','PCA7','PCA8','PCA19','PCA21','PCA33']

In [ ]:
X = arrowDF2[features_imp]
X.head()

,mfr,icc1,icc2,PCA0,PCA1,PCA3,PCA4,PCA5,PCA7,PCA8,PCA19,PCA21,PCA33
0,mfr_item0,icc1_item0,icc2_item0,-1.174946,-0.403428,0.215991,0.003821,-0.065177,0.299902,-0.040518,0.095916,0.014732,-0.003368
1,mfr_item0,icc1_item0,icc2_item0,-1.330728,-0.375666,0.218536,0.007240,-0.175244,0.099806,-0.100697,-0.052379,-0.082654,-0.008241
2,mfr_item0,icc1_item0,icc2_item0,-1.408653,-0.358471,0.231599,0.008366,-0.121095,0.445013,0.024451,0.012557,-0.087305,-0.004457
3,mfr_item0,icc1_item0,icc2_item0,-1.220221,-0.376939,0.214228,0.005685,-0.131327,0.206358,-0.051941,0.063039,-0.037352,-0.006275
6,mfr_item0,icc1_item0,icc2_item0,-1.433350,-0.359800,0.237394,0.006382,-0.080985,0.401849,0.069397,0.015012,-0.068194,-0.007333


In [ ]:
y = arrowDF2['actualleadtime'].copy()
y.head()

0    15
1     8
2    24
3    14
6     2
Name: actualleadtime, dtype: int64

In [ ]:
X_encoded = pd.get_dummies(X, columns=['icc1','icc2','mfr'])
X_encoded.head()
len(X_encoded)

39553

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, train_size=.80, test_size=.20, random_state=123) 

In [ ]:
print(len(X_train))
print(len(X_test))


31642
7911


## Explore: Basic Model Buildling, Trying Different Combos of Data/Variables

Base Model -- Kitchen Sink
Data: All, Variables: All except MFR, Split: 80/20, RMSE: 6.994, MAE: 5.532

In [ ]:
xgb_model1 = XGBRegressor()
xgb_model1.fit(X_train, y_train)

[22:27:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
preds = xgb_model1.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))
print("Mean Absolute Error : " + str(mean_absolute_error(preds, y_test)))

RMSE: 7.177400
Mean Absolute Error : 5.6423558216453005


Model Using Important Variables -- Scree -- Data: All excpet MFR, Split: 80/20
RMSE: 7.299736
MAE: 5.907082

In [ ]:
xgb_model_2 = XGBRegressor()
xgb_model_2.fit(X_train, y_train)

[22:27:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
preds_2 = xgb_model_2.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds_2))
print("RMSE: %f" % (rmse))
print("Mean Absolute Error : " + str(mean_absolute_error(preds_2, y_test)))

RMSE: 7.177400
Mean Absolute Error : 5.6423558216453005


1. Model Using Important Variables -- Imp_MFR_reduceddata -- Data: IMP MFR 60% excpet icc3, Split: 80/20 RMSE: 7.1774 MAE: 5.6435
Use this model, even though its a little worse than the base, theres less variables/noise 

In [ ]:
xgb_model_3 = XGBRegressor()
xgb_model_3.fit(X_train, y_train)

[22:27:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
preds_3 = xgb_model_3.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds_3))
print("RMSE: %f" % (rmse))
print("Mean Absolute Error : " + str(mean_absolute_error(preds_3, y_test)))

RMSE: 7.177400
Mean Absolute Error : 5.6423558216453005


Model Using Important Variables -- Used 1cc1, icc2, mfr and imp variables  -- Data: IMP MFR 60% excpet icc3, Split: 90/10 RMSE: 7.23 MAE: 5.71

In [ ]:
xgb_model_4 = XGBRegressor()
xgb_model_4.fit(X_train, y_train)

[22:28:06] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
preds_4 = xgb_model_4.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds_4))
print("RMSE: %f" % (rmse))
print("Mean Absolute Error : " + str(mean_absolute_error(preds_4, y_test)))

RMSE: 7.177400
Mean Absolute Error : 5.6423558216453005


Model Using Important Variables -- Used scree features  -- Data: IMP MFR 60% excpet icc3, Split: 90/10 RMSE: 7.20 MAE: 5.6

In [ ]:
xgb_model_5 = XGBRegressor()
xgb_model_5.fit(X_train, y_train)

[22:28:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.1, max_delta_step=0,
             max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
preds_5 = xgb_model_5.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds_5))
print("RMSE: %f" % (rmse))
print("Mean Absolute Error : " + str(mean_absolute_error(preds_5, y_test)))

RMSE: 7.177400
Mean Absolute Error : 5.6423558216453005


## 2. Hypertuning the parameters using RandomSearchCV


In [ ]:
param_tuning = {'learning_rate': [0.01, 0.03, 0.05, 0.07, 0.09, 0.1, 0.3],
                'max_depth': [3, 5, 7, 10],
                'n_estimators' : [100, 200, 500, 1000]
                }

gbm = xgb.XGBRegressor()

In [ ]:
# Perform random search: grid_mse
randomized_mse = RandomizedSearchCV(estimator=gbm, param_distributions=param_tuning,
                                    n_iter=10, scoring='neg_mean_squared_error', cv=4, verbose=1)
randomized_mse.fit(X_train, y_train)

Fitting 4 folds for each of 10 candidates, totalling 40 fits
[22:28:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[22:28:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:29:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:29:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:30:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:30:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:30:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:30:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[22:30:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed: 30.5min finished


[22:58:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


RandomizedSearchCV(cv=4, error_score=nan,
                   estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                          colsample_bylevel=1,
                                          colsample_bynode=1,
                                          colsample_bytree=1, gamma=0,
                                          importance_type='gain',
                                          learning_rate=0.1, max_delta_step=0,
                                          max_depth=3, min_child_weight=1,
                                          missing=None, n_estimators=100,
                                          n_jobs=1, nthread=None,
                                          objective='reg:linear',
                                          random_state=0, reg_alpha=0,
                                          reg_lambda=1, scale_pos_weight=1,
                                          seed=None, silent=None, subsample=1,
                                  

In [ ]:
# Print the best parameters and lowest RMSE
print("Best parameters found: ",randomized_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(randomized_mse.best_score_)))

Best parameters found:  {'n_estimators': 100, 'max_depth': 10, 'learning_rate': 0.07}
Lowest RMSE found:  6.910048736266091


Make predictions using the test data, should be better than the training metrics

## 3. Plug Optimal Values Found Above Into Model


In [ ]:
model_final = XGBRegressor(n_estimators=200, learning_rate=0.05, max_depth=10)
model_final.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_test, y_test)], 
             verbose=False)

[22:59:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=10, min_child_weight=1, missing=None, n_estimators=200,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [ ]:
preds_final = model_final.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds_final))
print("RMSE: %f" % (rmse))
print("Mean Absolute Error : " + str(mean_absolute_error(preds_final, y_test)))

RMSE: 6.807029
Mean Absolute Error : 5.236327932691954


Create final dataframe used for economic impact 

In [ ]:
X_test['Predictions'] = preds_final
X_test
X_test_short = X_test.iloc[:,-1:]
X_test_short

final_df_mini = X_test.join(arrowDF2['expected_leadtime'])
final_df = pd.concat([X_test_short, arrowDF2], axis=1, sort=False)
final_df.dropna(inplace=True)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
final_df = final_df[['Predictions', 'actualleadtime', 'expected_leadtime']]
final_df

,Predictions,actualleadtime,expected_leadtime
1,9.004291,8,20
17,3.317213,2,2
21,4.532260,2,2
22,2.033150,3,2
28,14.324252,22,10
...,...,...,...
65492,4.092854,2,2
65502,7.026614,4,2
65513,17.882050,14,16
65516,11.626758,14,16


## Economic Impact

In [ ]:
final_df['Observed_Difference']=final_df['actualleadtime']-final_df['expected_leadtime']
final_df['Predictions_Difference']=round(final_df['Predictions']-final_df['expected_leadtime'])
final_df['Predictions_Difference'].round()
final_df

,Predictions,actualleadtime,expected_leadtime,Observed_Difference,Predictions_Difference
1,9.004291,8,20,-12,-11.0
17,3.317213,2,2,0,1.0
21,4.532260,2,2,0,3.0
22,2.033150,3,2,1,0.0
28,14.324252,22,10,12,4.0
...,...,...,...,...,...
65492,4.092854,2,2,0,2.0
65502,7.026614,4,2,2,5.0
65513,17.882050,14,16,-2,2.0
65516,11.626758,14,16,-2,-4.0


In [ ]:
final_df['penalty'] = ""

for index, row in final_df.iterrows():
  
  
  if final_df.loc[index, 'Predictions_Difference'] < -2:
    final_df.loc[index, 'penalty'] = final_df.loc[index, 'Predictions_Difference'] * -400

  elif final_df.loc[index, 'Predictions_Difference'] <= 0 and final_df.loc[index, 'Predictions_Difference'] >= -2:
    final_df.loc[index, 'penalty'] = 0

  elif final_df.loc[index, 'Predictions_Difference'] == 1:
    final_df.loc[index, 'penalty'] = 400

  elif final_df.loc[index, 'Predictions_Difference'] >= 2 and final_df.loc[index, 'Predictions_Difference'] < 4:
    final_df.loc[index, 'penalty'] = 600

  elif final_df.loc[index, 'Predictions_Difference'] >= 4 and final_df.loc[index, 'Predictions_Difference'] < 8:
    final_df.loc[index, 'penalty'] = 1000

  else:
    final_df.loc[index, 'penalty'] = 4000




In [ ]:
# Economic Impact Model: Actual 

final_df['penalty_actual'] = ""

for index, row in final_df.iterrows():
  
  
  if final_df.loc[index, 'Observed_Difference'] < -2:
    final_df.loc[index, 'penalty_actual'] = final_df.loc[index, 'Observed_Difference'] * -400

  elif final_df.loc[index, 'Observed_Difference'] <= 0 and final_df.loc[index, 'Observed_Difference'] >= -2:
    final_df.loc[index, 'penalty_actual'] = 0

  elif final_df.loc[index, 'Observed_Difference'] == 1:
    final_df.loc[index, 'penalty_actual'] = 400

  elif final_df.loc[index, 'Observed_Difference'] >= 2 and final_df.loc[index, 'Observed_Difference'] < 4:
    final_df.loc[index, 'penalty_actual'] = 600

  elif final_df.loc[index, 'Observed_Difference'] >= 4 and final_df.loc[index, 'Observed_Difference'] < 8:
    final_df.loc[index, 'penalty_actual'] = 1000

  else:
    final_df.loc[index, 'penalty_actual'] = 4000



In [ ]:
# Validate that the predictions are correct 
final_df[final_df['Predictions_Difference'] == -20]

,Predictions,actualleadtime,expected_leadtime,Observed_Difference,Predictions_Difference,penalty,penalty_actual
203,10.202415,6,30,-24,-20.0,8000,9600
2612,9.506804,2,30,-28,-20.0,8000,11200
9221,9.364407,20,29,-9,-20.0,8000,3600
9710,5.673522,5,26,-21,-20.0,8000,8400
12221,4.028032,5,24,-19,-20.0,8000,7600
12843,9.506804,13,30,-17,-20.0,8000,6800
17970,5.996023,3,26,-23,-20.0,8000,9200
26337,15.677181,23,36,-13,-20.0,8000,5200
26967,11.371509,2,31,-29,-20.0,8000,11600
28409,11.235946,13,31,-18,-20.0,8000,7200


In [ ]:
penalty_diff = sum(final_df['penalty']) - sum(final_df['penalty_actual'])
penalty = sum(final_df['penalty'])
actual_penalty = sum(final_df['penalty_actual'])

In [ ]:
print("Penalty Predicted By Model: " + "{:,}".format(penalty))
print("Penalty From Actual Data: " + "{:,}".format(actual_penalty))
print("Penalty Difference: " + "{:,}".format(penalty_diff))

Penalty Predicted By Model: 15,289,400.0
Penalty From Actual Data: 18,577,400
Penalty Difference: -3,288,000.0
